In [ ]:
import test_parameter as p
import numpy as np
import pandas as pd
from PIL import Image 
import argparse
import func
import cv2
import random
import os

class Layer:
    def __init__(self):
        self.node = None
        self.num_node = p.num_hiddennode

class Node:
    def __init__(self):
        self.weight = []
        self.output_gradient = 0
        self.val = 0
        self.d_val = 0

def sigmoid(x):
    return np.round(1/(1+np.exp(-x)),8)


def LeakyReLu(x):
    return np.maximum(0.1 * x, x)

#init input, output layer
layer = [Layer() for _ in range(p.num_layer)]
layer[0].num_node = p.num_inputNode
layer[0].node = [Node() for _ in range(p.num_inputNode)]
layer[p.num_layer-1].num_node = p.num_outputNode

if func.check_arguement() == 0 :          
    #가중치 랜덤 초기화================================================
    w_temp = []
    for i in range(1,p.num_layer):
        layer[i].node = [Node() for _ in range(layer[i].num_node)]
        r_temp = np.sqrt(6. / layer[i-1].num_node)
        for j in range(layer[i].num_node):
            layer[i].node[j].weight = [round(random.uniform(-1 * r_temp, r_temp),8) for _ in range(layer[i-1].num_node)] 
            # 가중치 일반 초기화===========================================================================
            # layer[i].node[j].weight = [round(random.uniform(0,1),8) for _ in range(layer[i-1].num_node)]
            # ============================================================================================
            for k in range(len(layer[i].node[j].weight)):
                w_temp.append([i-1, k, i, j, layer[i].node[j].weight[k]])
    writer = pd.DataFrame(w_temp,columns=['f_layer', 'f_node', 't_layer', 't_node', 'weight'])
    writer.to_csv('test_weight.csv', index=False)
    #가중치 csv파일로 저장=============================================

image_list = os.listdir('./dataset/dog_fixed')

target = np.array([1,0,0,0])

for image_name in image_list:
    image_path = os.path.join('./dataset/dog_fixed', image_name)
    image = cv2.imread(image_path) 
    if image is not None:
        image = image.reshape(-1)
        image = image / 255.0
        for a in range(p.num_inputNode):
            layer[0].node[a].val = image[a]
    else:
        print(f"Failed to load image: {image_name}")
    
    #가중치 불러오기===================================================
    reader = pd.read_csv('test_weight.csv')
    idx = 0
    for i in range(1,p.num_layer):
        layer[i].node = [Node() for _ in range(layer[i].num_node)]
        for j in range(layer[i].num_node):
            layer[i].node[j].weight = [ 0 for _ in range(layer[i-1].num_node)]
            for k in range(len(layer[i].node[j].weight)):
                layer[i].node[j].weight[k] = reader.iloc[idx,4]
                idx += 1
    #가중치 불러오기 종료==============================================

    #순전파 시작
    #=================================================================
    for i in range(1,p.num_layer):
       
        prev_val = np.array([node.val for node in layer[i - 1].node])
        weights = np.array([node.weight for node in layer[i].node])
        z_val = np.round(np.dot(weights, prev_val),8)
        if i == p.num_layer - 1:
            
            exp_values = np.exp(z_val)
            sum_exp_values = np.sum(exp_values)
            outputs = exp_values / sum_exp_values
            print(outputs)
            print(np.sum(outputs))
            output_gradient = outputs * (1- outputs)
        else: 
            outputs = LeakyReLu(z_val)
            output_gradient = np.where(z_val > 0, 1, 0.1)
        # print("output_gradient")
        # print(output_gradient)

        for j, node in enumerate(layer[i].node):
            node.val = outputs[j]
            node.output_gradient = output_gradient[j]
    #순전파 종료
    #=================================================================

    #=================================================================
    result = layer[p.num_layer - 1]
    x = np.argmax([node.val for node in result.node])
    if x == 0: 
        print("이 그림은 '개' 입니다.")
    elif x == 1:
        print("이 그림은 '고양이' 입니다.")
    elif x == 2:
        print("이 그림은 '호랑이' 입니다.")
    elif x == 3:
        print("이 그림은 '하이에나' 입니다.")
    #=================================================================

    #역전파 시작
    #=================================================================
    #가중치 변화율 계산 시작
    for i in range(p.num_layer-1, 0, -1):
        if i == len(layer) - 1:  # 출력층
            # 출력층 노드의 기울기
            output = np.array([node.val for node in layer[i].node])
            d_val = (output - target)
            for j, node in enumerate(layer[i].node):
                node.d_val =  d_val[j] * layer[i].node[j].output_gradient
        else:  # 은닉층
            weights = np.transpose(np.array([node.weight for node in layer[i + 1].node]))
            prev_d_val = np.array([node.d_val for node in layer[i + 1].node])
            g_val = np.dot(weights, prev_d_val)
            #앞 레이어의 모든 노드의 d_val에 해당하는 가중치를 곱해 더한것이 g_val이자 node의 d_val
            for j, node in enumerate(layer[i].node):
                n, m = layer[i].node[j].output_gradient.size
                print(n,m)
                node.d_val = g_val[j] * layer[i].node[j].output_gradient


    #가중치 변화율 계산 종료
    #가중치 수정 시작
    for i in range(1,p.num_layer):
        for j in range(layer[i].num_node):
            for k in range(len(layer[i].node[j].weight)):
                layer[i].node[j].weight[k] -= p.LR * layer[i].node[j].d_val * layer[i-1].node[k].val
    #역전파 종료
    #=================================================================

    #가중치 기록 시작
    #=================================================================
    reader = pd.read_csv('test_weight.csv')
    idx = 0
    for i in range(1,p.num_layer):
        for j in range(layer[i].num_node):
            for k in range(len(layer[i].node[j].weight)):
                reader.iloc[idx,4] = layer[i].node[j].weight[k]
                idx += 1
    reader.to_csv('test_weight.csv', index=False)
    print("학습 끝")



